In [1]:
%run ../include/oracle.py

In [57]:
import random

#returns a random bitstring of length n
def random_bitstring(n):
    s = ""
    for i in range(n):
        s += random.choice(["0", "1"])  
    return s


def check_collision(f_dict):
    if len(f_dict.values()) != len(set(f_dict.values())):#If an element is repeated, then we found a collision
        return True
    return False
    

num_queries = [] # A list to hold the number of queries we performed at each iteration
for it in range(1000):
    f_dict = {} # This will hold the s and f(s) pairs. s is the key, f(s) is the value
    queries = 0
    found = False 
    while(not found): # Continue until finding a collision
        while True:
            s = random_bitstring(3)
            if s not in f_dict: #C ontinue until you pick an s which is not queried before
                break
        f_dict[s] = f(s) # We query f and store the result in f_dict
        queries += 1 # Increase the number of queries
        found = check_collision(f_dict) # Check if the f(s) = f(t) for different s and t
    num_queries.append(queries) # Store the number of queries made
    
print("Average number of queries is", sum(num_queries)/1000) #Print the average

Average number of queries is 3.501


In [51]:
random_bitstring(5)
# random_bitstring(2)

0
00
000
0000
00000


'00000'

In [56]:
import numpy as np
from scipy.sparse.linalg import eigsh

# Define the negated QUBO matrix (replace with your actual matrix if different)
Qp = np.array([[-2,  2,  2,  0,  0,  0],
               [ 0, -1,  0,  0,  0,  0],
               [ 0,  0, -3,  2,  2,  0],
               [ 0,  0,  0, -2,  2,  0],
               [ 0,  0,  0,  0,  0, -2]])

# Calculate minimum eigenvalue and eigenvector
lambda_min, v_min = eigsh(Qp, which='SM', k=1)

# Thresholding for binary approximation (adjust threshold as needed)
threshold = 0
x_approx = np.where(v_min < threshold, 0, 1)

print("Eigenvector (v_min):", v_min)
print("Approximated binary solution (x):", x_approx)


ValueError: expected square matrix (shape=(5, 6))

In [3]:
%run ../include/oracle.py
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

def simon():
    # Define quantum and classical registers. 
    qreg1 = QuantumRegister(3, "register_1")
    qreg2 = QuantumRegister(3, "register_2")
    creg = ClassicalRegister(3)

    # Define quantum Circuit
    simon_circuit = QuantumCircuit(qreg1, qreg2, creg)

    # Apply Hadamard to the first register
    simon_circuit.h(qreg1)
    simon_circuit.barrier()

    # Apply the simon_oracle function
    simon_circuit.compose(simon_oracle(), inplace=True)
    simon_circuit.barrier()
    
    simon_circuit.measure(qreg2,creg)
    simon_circuit.barrier()

    # Apply Hadamard to the first register
    simon_circuit.h(qreg1)
    simon_circuit.barrier()

    # Measure the first register
    simon_circuit.measure(qreg1,creg)
    
    job = AerSimulator().run(simon_circuit,shots=1)
    counts = job.result().get_counts()
    
    #Since the number of shots is 1, there is only a single key and we return it
    return list(counts.keys())[0]

In [6]:
n = 3
y_list = []
it = 1
while(len(y_list)) < n-1:
    y = simon() 
    if y!='000' and y not in y_list: #Omit 000 string omit if y already exists in the list
        y_list.append(y) #we get the key and append it to our list
    it+=1
print(y_list)
print(f"It took {it} iterations to obtain the list of y's")

['001', '101']
It took 8 iterations to obtain the list of y's
